<a href="https://colab.research.google.com/github/Antoine-Cohen/PI2_Hearme/blob/Machine-Learning-models/Essai_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_io

     |████████████████████████████████| 23.1 MB 1.2 MB/s 
     |████████████████████████████████| 2.1 MB 39.2 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.22.0
    Uninstalling tensorflow-io-gcs-filesystem-0.22.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.22.0


In [ ]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
testing_wav_file_name = tf.keras.utils.get_file('cough.wav',
                                                'https://storage.googleapis.com/audioset/yamalyzer/audio/cough.wav',
                                                cache_dir='./',
                                                cache_subdir='test_data')

print(testing_wav_file_name)

409600/395088 [===============================] - 0s 0us/step
./test_data/cough.wav


In [ ]:
# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
testing_wav_data = load_wav_16k_mono(testing_wav_file_name)

In [ ]:
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

In [ ]:
_ = tf.keras.utils.get_file('esc-50.zip',
                        'https://github.com/karoldvl/ESC-50/archive/master.zip',
                        cache_dir='./',
                        cache_subdir='datasets',
                        extract=True)

645693440/Unknown - 47s 0us/step

In [ ]:
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
base_data_path = './datasets/ESC-50-master/audio/'

pd_data = pd.read_csv(esc50_csv)
pd_data.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [ ]:
cat_list = ['dog','cat','vacuum_cleaner','thunderstorm','washing-machine','car_horn','alarm_clock','train','airplane','coughing']
map_class_to_id = dict(zip(cat_list,np.arange(0,len(cat_list),1)))
filtered_pd = pd_data[pd_data.category.isin(cat_list)]

class_id = filtered_pd['category'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(target=class_id)

full_path = filtered_pd['filename'].apply(lambda row: os.path.join(base_data_path, row))
filtered_pd = filtered_pd.assign(filename=full_path)

filtered_pd.head(10)

,filename,fold,target,category,esc10,src_file,take
0,./datasets/ESC-50-master/audio/1-100032-A-0.wav,1,0,dog,True,100032,A
2,./datasets/ESC-50-master/audio/1-100210-A-36.wav,1,2,vacuum_cleaner,False,100210,A
3,./datasets/ESC-50-master/audio/1-100210-B-36.wav,1,2,vacuum_cleaner,False,100210,B
4,./datasets/ESC-50-master/audio/1-101296-A-19.wav,1,3,thunderstorm,False,101296,A
5,./datasets/ESC-50-master/audio/1-101296-B-19.wav,1,3,thunderstorm,False,101296,B
14,./datasets/ESC-50-master/audio/1-110389-A-0.wav,1,0,dog,True,110389,A
16,./datasets/ESC-50-master/audio/1-115521-A-19.wav,1,3,thunderstorm,False,115521,A
25,./datasets/ESC-50-master/audio/1-11687-A-47.wav,1,8,airplane,False,11687,A
28,./datasets/ESC-50-master/audio/1-119125-A-45.wav,1,7,train,False,119125,A
53,./datasets/ESC-50-master/audio/1-17124-A-43.wav,1,5,car_horn,False,17124,A


In [ ]:
filenames = filtered_pd['filename']
targets = filtered_pd['target']
folds = filtered_pd['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 4)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 4)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 5)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
my_model = tf.keras.Sequential([                      
        tf.keras.layers.Conv1D(32,(3),activation='relu',input_shape = (1024,1)),
        tf.keras.layers.MaxPooling1D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Conv1D(64,(3),activation='relu'),
        tf.keras.layers.MaxPooling1D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
                                                       
])

In [ ]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                            patience=3,
                                            restore_best_weights=True)

In [ ]:
history = my_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


     60/Unknown - 5s 74ms/step - loss: 1.0849 - accuracy: 0.7078WARNING:tensorflow:Early stopping conditioned on metric `accuracy*val_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


60/60 [==============================] - 6s 88ms/step - loss: 1.0849 - accuracy: 0.7078 - val_loss: 0.6127 - val_accuracy: 0.8172
Epoch 2/20
60/60 [==============================] - ETA: 0s - loss: 0.5495 - accuracy: 0.8036WARNING:tensorflow:Early stopping conditioned on metric `accuracy*val_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


60/60 [==============================] - 5s 83ms/step - loss: 0.5495 - accuracy: 0.8036 - val_loss: 0.4859 - val_accuracy: 0.8000
Epoch 3/20
60/60 [==============================] - ETA: 0s - loss: 0.5089 - accuracy: 0.8224WARNING:tensorflow:Early stopping conditioned on metric `accuracy*val_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


60/60 [==============================] - 5s 83ms/step - loss: 0.5089 - accuracy: 0.8224 - val_loss: 0.4956 - val_accuracy: 0.8359
Epoch 4/20
60/60 [==============================] - ETA: 0s - loss: 0.4293 - accuracy: 0.8339WARNING:tensorflow:Early stopping conditioned on metric `accuracy*val_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


60/60 [==============================] - 5s 83ms/step - loss: 0.4293 - accuracy: 0.8339 - val_loss: 0.5154 - val_accuracy: 0.8000


KeyboardInterrupt: ignored

In [ ]:
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

20/20 [==============================] - 1s 27ms/step - loss: 1.1191 - accuracy: 0.7484
Loss:  1.1191489696502686
Accuracy:  0.7484375238418579
